In [2]:
#installig kaggle library
! pip install kaggle library

  Preparing metadata (setup.py) ... done
  Created wheel for library: filename=Library-0.0.0-py3-none-any.whl size=2055 sha256=6b0803f20906b654b94367cb764865cb36c53797d1d3bd58bf5df29508b4dfbf
  Stored in directory: /root/.cache/pip/wheels/e0/71/7d/b0e29b944e43374597cd4e3b88c85197001c9bfcd5dce191f4
Successfully built library


In [3]:
#configuring the path of kaggle.json file
!mdkir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

/bin/bash: line 1: mdkir: command not found
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
 # API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
100% 80.9M/80.9M [00:00<00:00, 160MB/s]
100% 80.9M/80.9M [00:00<00:00, 152MB/s]


In [5]:
#extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset is extracted')



the dataset is extracted


In [6]:
import numpy as np
import pandas as pd
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [7]:
 # JUST AN EXAMPLE TO HOW TO USE PORTERSTEMMER

from nltk.stem.porter import PorterStemmer
word = "running"
stemmer = PorterStemmer()  #CREATING THE OBJECT OF PORTERSTEMMER
stemmed_word = stemmer.stem(word)
stemmed_word

'run'

In [8]:
import nltk
nltk.download('stopwords') # download the stopwords from nltk library.
stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


<WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>

In [9]:
print(stopwords.words('english')) #printing the stopwords in language english


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PROCESSING

In [10]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'ISO-8859-1')  # encoding is the transfer of information into a special format

In [11]:
# checking the number of rows and columns
twitter_data.shape

(1599999, 6)

In [12]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
# nmaing the columns and reading the datasets again
column_names = ['target','ids','date' , 'flag','user','text']
column_names

['target', 'ids', 'date', 'flag', 'user', 'text']

In [14]:
# creating the columns name beacuse ist row read as data not column
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' ,  names = column_names ,encoding = 'ISO-8859-1')


In [15]:
# now you see the number of rows increase
twitter_data.shape

(1600000, 6)

In [16]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
# counting the number of missing values or text .
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [18]:
# checking the distribution of target columns

twitter_data['target'].value_counts()

#we check this because we have to ensure that data distribution  is in even form if data  distribution is not in even form then we have to consider it as a problem



,count
target,
0,800000
4,800000


convert the 'target' "4" to "1"


In [19]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [20]:
# now we  check that 'target' is relpaced or not by above code .
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


'0' means negative tweet
'1' means positive tweet


### ***STEMMING***

It is the process of reducing a word to its Root word . i mentioned a clear example on that above in "6th cell"

Another example : actor , actress , acting = "**ACT**"

In [21]:
def stemming(content):
  stemmed_content = re.sub('[a-zA-Z]', ' ' , content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  port = PorterStemmer()
  stemmed_content = [port.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [22]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [23]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@ :// . /2 1 - , ' . . ;"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,' ... . !
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@ . 50%
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@ , ' . ' . ? ' ."


In [25]:
print(twitter_data['stemmed_content'])

0          @ :// . /2 1 - , ' . . ;
1                         ' ... . !
2                           @ . 50%
3                                  
4                 @ , ' . ' . ? ' .
                     ...           
1599995                           .
1599996         . - ! â« :// . /~8
1599997                           ?
1599998                      38 !!!
1599999                   # @ @ @ 4
Name: stemmed_content, Length: 1600000, dtype: object


In [26]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [27]:
#seperating the data and label
X= twitter_data['stemmed_content']
Y= twitter_data['target']

In [28]:
print(X)

0          @ :// . /2 1 - , ' . . ;
1                         ' ... . !
2                           @ . 50%
3                                  
4                 @ , ' . ' . ? ' .
                     ...           
1599995                           .
1599996         . - ! â« :// . /~8
1599997                           ?
1599998                      38 !!!
1599999                   # @ @ @ 4
Name: stemmed_content, Length: 1600000, dtype: object


In [29]:
print(Y)

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


Splitting the data to training data and test data


In [30]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [34]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)
(1600000,) (1280000,) (320000,)


In [35]:
print(X_train)

1570269          
1273074     @ ' !
88479            
254604         ,,
667941        . .
            ...  
941805     @ ....
1007131         @
1460311          
929226      ! ' !
526253        . .
Name: stemmed_content, Length: 1280000, dtype: object


In [36]:
print(X_test)

131348     @ - ! ' . & ; .
1142114              / & ;
244564                 @ !
445353               ... !
415893                    
                ...       
178459     ... !! ... ' !!
1515130                @ '
1449952      @ 41 ! 4700 !
441063                    
1583304           @ !!!! !
Name: stemmed_content, Length: 320000, dtype: object


Feature Extraction


In [38]:
#converrting the textual data to numericaldata
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [39]:
print(X_train)

  (5, 3469)	1.0
  (6, 8484)	0.7382151603694854
  (6, 8444)	0.6745653244872989
  (20, 2318)	1.0
  (30, 533)	1.0
  (45, 1447)	1.0
  (46, 740)	1.0
  (61, 739)	1.0
  (74, 5784)	1.0
  (88, 6276)	1.0
  (96, 4499)	0.8008755702940765
  (96, 4235)	0.5988307948879529
  (99, 4055)	1.0
  (106, 7028)	1.0
  (107, 2534)	1.0
  (108, 4318)	1.0
  (109, 1784)	0.75753502473072
  (109, 1)	0.6527945207385148
  (110, 6772)	1.0
  (123, 4825)	0.5532665234794572
  (123, 4773)	0.8330042941047515
  (136, 739)	1.0
  (139, 5644)	1.0
  (142, 6127)	1.0
  (149, 3921)	0.6436322420464065
  :	:
  (1279826, 6920)	1.0
  (1279836, 7608)	1.0
  (1279842, 5938)	1.0
  (1279843, 5062)	1.0
  (1279848, 789)	1.0
  (1279849, 3142)	1.0
  (1279851, 3550)	1.0
  (1279861, 1656)	1.0
  (1279863, 4476)	1.0
  (1279864, 2686)	1.0
  (1279880, 2687)	1.0
  (1279894, 1207)	0.5927335051951225
  (1279894, 304)	0.8053986539714999
  (1279900, 4536)	1.0
  (1279913, 5551)	1.0
  (1279914, 3047)	1.0
  (1279917, 739)	1.0
  (1279934, 7455)	0.5615353500148

In [40]:
print(X_test)

  (6, 7029)	1.0
  (15, 2470)	1.0
  (19, 1599)	1.0
  (20, 6667)	1.0
  (23, 3552)	1.0
  (37, 2470)	1.0
  (41, 6968)	1.0
  (54, 902)	1.0
  (61, 6074)	1.0
  (64, 739)	1.0
  (66, 3613)	0.7178771988917738
  (66, 2534)	0.45491671156885005
  (66, 377)	0.5269754385611195
  (78, 1447)	1.0
  (92, 6641)	1.0
  (110, 8270)	1.0
  (113, 974)	1.0
  (114, 4055)	1.0
  (123, 3550)	1.0
  (131, 8270)	1.0
  (149, 4655)	1.0
  (170, 740)	1.0
  (176, 740)	1.0
  (184, 739)	1.0
  (195, 6772)	1.0
  :	:
  (319773, 659)	1.0
  (319822, 7976)	1.0
  (319834, 3550)	1.0
  (319844, 2504)	1.0
  (319849, 6825)	1.0
  (319860, 2504)	0.6699573225007557
  (319860, 0)	0.7423996134344486
  (319862, 6276)	1.0
  (319878, 4234)	0.7759385055112661
  (319878, 3550)	0.6308085570638233
  (319879, 6362)	1.0
  (319887, 739)	1.0
  (319898, 5946)	1.0
  (319907, 3306)	1.0
  (319917, 3469)	1.0
  (319922, 5391)	1.0
  (319929, 825)	1.0
  (319941, 8206)	1.0
  (319952, 1803)	1.0
  (319961, 6565)	1.0
  (319970, 1447)	1.0
  (319982, 3140)	1.0
  (31

Training the machine learning model


Logistic regression


In [41]:
model = LogisticRegression(max_iter=1000)

In [42]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score


In [43]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [46]:
print('Accuracy score on the the training data : ',training_data_accuracy)

Accuracy score on the the training data :  0.516021875


In [47]:
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [48]:
print('Accuracy score on the the testing data : ',testing_data_accuracy)

Accuracy score on the the testing data :  0.510915625


Saving the trained model


In [49]:
import pickle

In [50]:
filename = 'trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the saved model for future prediction


In [51]:
  #loading the saved model
  loaded_model = pickle.load(open('trained_model.sav','rb'))

In [66]:
X_new = X_test[200]
print(Y_test.iloc[200])
prediction =loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Neagtive Tweet')
else:
  print('Positive Tweet')

1
[0]
Neagtive Tweet


In [69]:
X_new = X_test[3]
print(Y_test.iloc[3])
prediction =loaded_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('Neagtive Tweet')
else:
    print('Positive Tweet')

0
[0]
Neagtive Tweet
